# TIMES BETWEEN STOPS ANALYSIS

In [1]:
import pandas as pd
pd.set_option("display.precision", 3)
import numpy as np
import json

import datetime
from datetime import timedelta

import statistics
from statistics import mean
import math

from pandarallel import pandarallel
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
pandarallel.initialize()

import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
pio.templates.default = 'plotly_white'

from scipy import stats
from statsmodels.stats import weightstats as stests

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## Lines collected dictionary

In [2]:
#Line collected dict
with open('M6Data/lines_collected_dict.json', 'r') as f:
    lines_collected_dict = json.load(f)

## Buses time intervals differentiation
From the data available for each line, we are going to construct a dictionary with the time intervals for each line
and the frequencies of the buses for that intervals

In [3]:
#Line stops dict
with open('M6Data/freq_ranges_dict.json', 'r') as f:
    freq_ranges_dict = json.load(f)

In [4]:
day_type_dict = { #0 = Monday, 1 = Tuesday ...
    'LA' : [0,1,2,3,4], #LABORABLES
    'LJ' : [0,1,2,3], #LUNES A JUEVES
    'VV' : [4], #VIERNES
    'SA' : [5], #SABADOS
    'FE' : [6], #DOMIGOS O FESTIVOS
}

In [5]:
#Coronavirus Interval
covid_start = datetime.datetime(2020,3,16)
covid_end = datetime.datetime.now()

## Times between stops for each time interval
Here we are going to calculate the mean time between stops for each time interval of the day, for the data recovered, making use of the preprocessed attribute 'arrival_time'. This data is calculated in a script called 'times_bt_stops.py' and loaded here.

In [6]:
#Load times between stops data
times_bt_stops = pd.read_csv('../../flash/EMTBuses/ProcessedData/times_bt_stops.csv',
    dtype={
        'line': 'str',
        'direction': 'uint16',
        'st_hour': 'uint16',
        'end_hour': 'uint16',
        'stopA': 'uint16',
        'stopB': 'uint16',
        'bus': 'uint16',
        'trip_time':'float32',
        'api_trip_time':'int32'
    }
)[['line','direction','date','st_hour','end_hour','stopA','stopB','bus','trip_time','api_trip_time']]
#Parse the dates
times_bt_stops['date'] = pd.to_datetime(times_bt_stops['date'], format='%Y-%m-%d')
times_bt_stops.head()

,line,direction,date,st_hour,end_hour,stopA,stopB,bus,trip_time,api_trip_time
0,1,1,2020-02-25,20,21,4514,4022,123,86.204,114
1,1,1,2020-02-25,20,21,4022,3687,123,42.723,51
2,1,1,2020-02-25,20,21,4022,3687,8324,135.143,59
3,1,1,2020-02-25,20,21,3687,737,123,96.985,67
4,1,1,2020-02-25,20,21,737,735,123,71.892,62


In [18]:
#Select covid and no covid data

#Selection attributes
line = '132'
direction = 1
day_type = 'LA'
time_int = 1
stopA = int(lines_collected_dict[line][str(direction)]['stops'][1])
stopB = int(lines_collected_dict[line][str(direction)]['stops'][2])
st_hour = 7
end_hour = 11

#Select data
sel_df = times_bt_stops.loc[(times_bt_stops.line == line) & \
                      (times_bt_stops.direction == direction) & \
                      (times_bt_stops.date.dt.weekday.isin(day_type_dict[day_type])) & \
                      (times_bt_stops.stopA == stopA) & (times_bt_stops.stopB == stopB) & \
                      (times_bt_stops.st_hour >= st_hour) & (times_bt_stops.end_hour <= end_hour)]

tbs = sel_df.loc[(sel_df.date < covid_start)]
tbs_covid = sel_df.loc[(sel_df.date > covid_start)]

## COMPARISON OF TIMES BETWEEN STOPS BEFORE AND AFTER CORONAVIRUS

In [19]:
#Comparison of data
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=tbs.trip_time,name='Before COVID-19',
    histnorm='probability',
    xbins=dict(
        size=5
    )
))
fig.add_trace(go.Histogram(
    x=tbs_covid.trip_time,name='While COVID-19',
    histnorm='probability',
    xbins=dict(
        size=10
    )
))
# Overlay both histograms
fig.update_layout(
    title='Lapsed Time from stop {} to {} of line {} in {} days. {}:00-{}:00.'.format(stopA,stopB,line,day_type,st_hour,end_hour),
    xaxis_title='Seconds',
    yaxis_title='Normalized Count'
)
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()

In [9]:
#Comparison of data
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=tbs.api_trip_time,name='Before COVID-19',
    histnorm='probability',
    xbins=dict(
        size=5
    )
))
fig.add_trace(go.Histogram(
    x=tbs_covid.api_trip_time,name='While COVID-19',
    histnorm='probability',
    xbins=dict(
        size=10
    )
))
# Overlay both histograms
fig.update_layout(
    title='API Time from stop {} to {} of line {} in {} days. {}:00-{}:00.'.format(stopA,stopB,line,day_type,st_hour,end_hour),
    xaxis_title='Seconds',
    yaxis_title='Normalized Count'
)
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()

In [10]:
#API vs Lapsed data
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=tbs.trip_time,name='Lapsed time - Before COVID-19',
    histnorm='probability',
    xbins=dict(
        size=5
    )
))
fig.add_trace(go.Histogram(
    x=tbs.api_trip_time,name='API time - Before COVID-19',
    histnorm='probability',
    xbins=dict(
        size=10
    )
))
# Overlay both histograms
fig.update_layout(
    title='API Time from stop {} to {} of line {} in {} days. {}:00-{}:00.'.format(stopA,stopB,line,day_type,st_hour,end_hour),
    xaxis_title='Seconds',
    yaxis_title='Normalized Count'
)
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()

In [11]:
#API vs Lapsed data
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=tbs_covid.trip_time,name='Lapsed time - While COVID-19',
    histnorm='probability',
    xbins=dict(
        size=5
    )
))
fig.add_trace(go.Histogram(
    x=tbs_covid.api_trip_time,name='API time - While COVID-19',
    histnorm='probability',
    xbins=dict(
        size=10
    )
))
# Overlay both histograms
fig.update_layout(
    title='API Time from stop {} to {} of line {} in {} days. {}:00-{}:00.'.format(stopA,stopB,line,day_type,st_hour,end_hour),
    xaxis_title='Seconds',
    yaxis_title='Normalized Count'
)
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()

## Two samples Z test to check if the data before COVID is different to the data while it

In [12]:
N = min(tbs.shape[0],tbs_covid.shape[0])

#Test data ramdom sample size N selection
test_data1 = tbs
test_data2 = tbs_covid

#test_data1 = tbs.sample(N) 
#test_data2 = tbs_covid.sample(N)

#Mean to test
mean_to_test = test_data1.trip_time.mean()

print('Null Hypothesis : Mean travel time during the coronavirus is equal to the mean travel time before it')

#Hyp. test
ztest ,pval1 = stests.ztest(x1=test_data2.trip_time, value=mean_to_test,alternative='two-sided')
print('p-Value = {}'.format(pval1))
if pval1 < 0.05 : 
    print('Null hypothesis rejected.')
else :
    print('Null hyphotesis is not rejected.')

Null Hypothesis : Mean travel time during the coronavirus is equal to the mean travel time before it
p-Value = 4.118564295388477e-56
Null hypothesis rejected.


In [13]:
print('Null hypothesis : Mean travel time of the API is adapted to COVID status')
#Mean to test 
mean_to_test = test_data2.trip_time.mean()

#Hyp. test
ztest ,pval1 = stests.ztest(x1=test_data2.api_trip_time, value=mean_to_test,alternative='two-sided')
print('p-Value = {}'.format(pval1))
if pval1 < 0.05 : 
    print('Null hypothesis rejected.')
else :
    print('Null hyphotesis is not rejected.')

Null hypothesis : Mean travel time of the API is adapted to COVID status
p-Value = 7.411058000694918e-12
Null hypothesis rejected.


In [14]:
print('Null hypothesis : API mean travel time is the same as the one before COVID-19')
#Mean to test 
mean_to_test = test_data1.api_trip_time.mean()

#Hyp. test
ztest ,pval1 = stests.ztest(x1=test_data2.api_trip_time, value=mean_to_test,alternative='two-sided')
print('p-Value = {}'.format(pval1))
if pval1 < 0.05 : 
    print('Null hypothesis rejected.')
else :
    print('Null hyphotesis is not rejected.')

Null hypothesis : API mean travel time is the same as the one before COVID-19
p-Value = 1.279047289338141e-33
Null hypothesis rejected.


# NOW THAT WE  HAVE PROVED THAT COVID 19 HIGHLY AFFECTS THE TIMES BT STOPS, WE ARE GOING TO ANALYSE ONLY THE DATA COLLECTED BEFORE IT

## TIMES BT STOPS DEPENDING ON DAY HOUR

In [15]:
#Rainbow set of colors
N = 24
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]
#Selection options
line = '1'
direction = 1
day_type = 'LA'
stopA = int(lines_collected_dict[line][str(direction)]['stops'][5])
stopB = int(lines_collected_dict[line][str(direction)]['stops'][6])

#Select data
sel_df = times_bt_stops.loc[(times_bt_stops.line == line) & \
                      (times_bt_stops.direction == direction) & \
                      (times_bt_stops.date.dt.weekday.isin(day_type_dict[day_type])) & \
                      (times_bt_stops.stopA == stopA) & (times_bt_stops.stopB == stopB)]
sel_df = sel_df.loc[sel_df.date<covid_start]

#Create figure
fig = go.Figure()
#Hour selection and traces
hour_dfs = []
for i in range(N) :
    hour_df = sel_df.loc[sel_df.st_hour == i]
    hour_dfs.append(hour_df)
    length = hour_df.shape[0]
    if length > 0 :
        x1 = ['{}:00'.format(i) for elem in range(length)]
        x_real = ['Lapsed' for elem in range(length)]
        x_API = ['API' for elem in range(length)]
        #Add hour trace
        fig.add_trace(go.Box(
            x=[x1,x_real],
            y=hour_df.trip_time,
            marker_color=c[i]
        ))
        fig.add_trace(go.Box(
            x=[x1,x_API],
            y=hour_df.api_trip_time,
            marker_color=c[i]
        ))

#Layout
fig.update_layout(
    title='Time from stop {} to {} of line {} in {} days'.format(stopA,stopB,line,day_type),
    showlegend=False,
    xaxis=dict(title='Start Hour'),
    yaxis=dict(title='Seconds',range=[0,400]),
)

fig.show()